In [ ]:
!pip install cvxpy
!pip install pandas
!pip install scikit-learn
!pip install numpy
!pip install scipy
!pip install mosek

In [1]:
import cvxpy as cvx
cvx.installed_solvers()

['ECOS', 'ECOS_BB', 'MOSEK', 'OSQP', 'SCIPY', 'SCS']

<h1>Select a Dataset</h1>

In [7]:
import numpy as np
import pandas as pd

DATASET = 'datasets/iris.csv'

df = pd.read_csv(DATASET, index_col=False)

n = len(df)

print(df)

data = df.iloc[:, :4].values
output = df.iloc[:, 4].values

K = len(np.unique(output))

     sepal-length  sepal-width  petal-length  petal-width      class
0             5.1          3.5           1.4          0.2     setosa
1             4.9          3.0           1.4          0.2     setosa
2             4.7          3.2           1.3          0.2     setosa
3             4.6          3.1           1.5          0.2     setosa
4             5.0          3.6           1.4          0.2     setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  virginica
146           6.3          2.5           5.0          1.9  virginica
147           6.5          3.0           5.2          2.0  virginica
148           6.2          3.4           5.4          2.3  virginica
149           5.9          3.0           5.1          1.8  virginica

[150 rows x 5 columns]


In [8]:
import numpy as np
import pandas as pd

DATASET = 'datasets/wine.csv'

df = pd.read_csv(DATASET, index_col=None, header=None)

n = len(df)

print(df)

data = df.iloc[:, 1:].values
output = df.iloc[:, 0].values

K = len(np.unique(output))

     0      1     2     3     4    5     6     7     8     9      10    11   
0     1  14.23  1.71  2.43  15.6  127  2.80  3.06  0.28  2.29   5.64  1.04  \
1     1  13.20  1.78  2.14  11.2  100  2.65  2.76  0.26  1.28   4.38  1.05   
2     1  13.16  2.36  2.67  18.6  101  2.80  3.24  0.30  2.81   5.68  1.03   
3     1  14.37  1.95  2.50  16.8  113  3.85  3.49  0.24  2.18   7.80  0.86   
4     1  13.24  2.59  2.87  21.0  118  2.80  2.69  0.39  1.82   4.32  1.04   
..   ..    ...   ...   ...   ...  ...   ...   ...   ...   ...    ...   ...   
173   3  13.71  5.65  2.45  20.5   95  1.68  0.61  0.52  1.06   7.70  0.64   
174   3  13.40  3.91  2.48  23.0  102  1.80  0.75  0.43  1.41   7.30  0.70   
175   3  13.27  4.28  2.26  20.0  120  1.59  0.69  0.43  1.35  10.20  0.59   
176   3  13.17  2.59  2.37  20.0  120  1.65  0.68  0.53  1.46   9.30  0.60   
177   3  14.13  4.10  2.74  24.5   96  2.05  0.76  0.56  1.35   9.20  0.61   

       12    13  
0    3.92  1065  
1    3.40  1050  
2    3.17

In [25]:
import pandas as pd
from scipy.io import loadmat

DATASET = 'datasets/mnist.mat'
mnist = loadmat(DATASET)
mnist_data = mnist["data"].T
mnist_label = mnist["label"][0]

print(mnist_data.shape)

labels = pd.DataFrame(mnist_label, columns=['label'])
digits = pd.DataFrame(mnist_data, columns=[f'{i}' for i in range(784)])

dfo = pd.concat([labels, digits], axis = 1)

n = 100
K = 5

dfs = [None for _ in range(K)]
for i in range(K):
    dfs[i] = dfo.loc[dfo['label'] == i]
    
df = pd.DataFrame()
for i in range(K):
    df = pd.concat([df, dfs[i].sample(n // K)])

df.reset_index(inplace=True, drop=True)
print(df)

data = df.iloc[:n, 1:].values / 255
output = df.iloc[:n, 0].values

(70000, 784)
    label  0  1  2  3  4  5  6  7  8  ...  774  775  776  777  778  779  780   
0     0.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0  \
1     0.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
2     0.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
3     0.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
4     0.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
..    ... .. .. .. .. .. .. .. .. ..  ...  ...  ...  ...  ...  ...  ...  ...   
95    4.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
96    4.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
97    4.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
98    4.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   
99    4.0  0  0  0  0  0  0  0  0  0  ...    0    0    0    0    0    0    0   

    781  782  783  
0     

<h1>Run Algorithm</h1>

In [26]:
import cvxpy as cvx
import numpy as np
import time

def norm(a, b):
    return np.sqrt(np.sum(np.power(np.subtract(a, b), 2)))


def weight(a, b):
    return norm(a, b)


def get_adj_matrix(data):
    n = len(data)
    adj_mat = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i != j:
                dist = weight(data[i], data[j])
                adj_mat[i, j] = adj_mat[j, i] = dist

    return adj_mat


def max_k_cut(raw_data, k):
    n = len(raw_data)
    dim = len(raw_data[0])
    data = get_adj_matrix(raw_data)

    Y = cvx.Variable((n, n), PSD=True)

    constraints = []
    for i in range(n):
        constraints.append(Y[i, i] == 1)
        for j in range(n):
            if i != j:
                constraints.append(Y[i, j] >= (-1 / (k - 1)))

    expr = cvx.sum(cvx.multiply(data, np.ones((n, n)) - Y))

    problem = cvx.Problem(cvx.Maximize(expr), constraints)
    problem.solve(solver='MOSEK')

    y = Y.value

    return y

def fixed_point(x, k, n):
    A = ((1 - k/2) / (k-1)) * np.ones(n)
        
    Y = cvx.Variable((n, n), PSD=True)

    constraints = []
    for l in range(n):
        constraints.append(Y[l, l] == 1)
        for j in range(n):
            if l != j:
                constraints.append(Y[l, j] >= (-1 / (k - 1)))
                
    expr = cvx.sum(cvx.multiply(x + A, Y))
    
    problem = cvx.Problem(cvx.Maximize(expr), constraints)
    problem.solve(solver='MOSEK')
    
    y = Y.value
    
    return y

In [ ]:
from sklearn.metrics import adjusted_rand_score, rand_score

x = max_k_cut(data, K)
print('SDP Matrix Built')

mat = np.reshape(x, (n, n))

a = -1 / (K - 1)

MIN_ERROR = 1E-4

i = 1
while True:
  new_x = fixed_point(x, K, n)
  max_diff = np.max(np.abs(new_x - x))
  x = new_x

  print(f'Fixed Point Iteration #{i} Complete (Maximum Change = {max_diff})')
  if (max_diff < MIN_ERROR):
      break
  i += 1

print('Fixed Point Iteration Complete')

unused = [i for i in range(n)]
classifier = [-1 for i in range(n)]

for i in range(K):
    current = unused[0]

    j = 0

    while j < len(unused):
        check_a = x[current, unused[j]]
        is_same = round(check_a * (K - 1)) // (K - 1)

        if is_same == 1:
            classifier[unused[j]] = i
            unused.remove(unused[j])
        else:
            j += 1

print('Classifier:', classifier)
print('Rand Index:', rand_score(output, classifier))
print('Adjusted Rand Index:', adjusted_rand_score(output, classifier))

SDP Matrix Built
